In [1]:
import sqlite3
import requests
import logging

from itertools import chain, islice
from collections import defaultdict


In [2]:
sqlite_path = 'test.db'
con = sqlite3.connect(sqlite_path)
cursor = con.cursor()
logging.basicConfig(level= logging.DEBUG)


In [3]:
def _create_tables():
    logging.debug("Create Tables")
    country_table="""
    CREATE TABLE IF NOT EXISTS country(
    id integer PRIMARY KEY AUTOINCREMENT,
    country text
    )
    """
    product_table ="""
    CREATE TABLE IF NOT EXISTS petroleum_product(
    id integer PRIMARY KEY AUTOINCREMENT,
    product_name text
    )
    """
    petroleum_sales="""
    CREATE TABLE IF NOT EXISTS petroleum_sales(
    id serial PRIMARY KEY ,
    country_id,
    product_id,
    year interger,
    sales float,
    FOREIGN KEY(country_id) REFERENCES country(id),
    FOREIGN KEY(product_id) REFERENCES petroleum_product(id)
    )
    """
    cursor.execute(country_table)
    cursor.execute(product_table)
    cursor.execute(petroleum_sales)
    
    con.commit()
    logging.debug("Table creation complete!")

_create_tables()

DEBUG:root:Create Tables
DEBUG:root:Table creation complete!


In [4]:
#Loading the data to the database
url="https://raw.githubusercontent.com/younginnovations/internship-challenges/master/programming/petroleum-report/data.json"

def _get_data(url):
    raw_data = requests.get(url)
    return raw_data.json()

raw_dict = _get_data(url)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:443
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "GET /younginnovations/internship-challenges/master/programming/petroleum-report/data.json HTTP/1.1" 200 1848


In [109]:
# cursor.execute("drop table if exists country")
# cursor.execute("drop table if exists petroleum_product")

In [115]:
# country_names = set([(x['country'],) for x in raw_dict]) #to insert we need list of tuples
# print(list(country_names))

[('Russia',), ('USA',), ('Saudi Arabia',), ('Isereal',)]


In [5]:
def _populate_country_table(raw_dict): 
    country_names = set([(x['country'],) for x in raw_dict]) #if tuple with length=1, we need comma 

    cursor.executemany("INSERT INTO COUNTRY(country) VALUES (?)",list(country_names))
    cursor.execute("SELECT * FROM country")

    country_with_ids = cursor.fetchall() #returns all the rows as a list of tuples

    return {x[1]: x[0] for x in country_with_ids} #returns all rows as a dictionary

country_with_id = _populate_country_table(raw_dict)


In [6]:
print(country_with_id)

{'USA': 1, 'Russia': 2, 'Saudi Arabia': 3, 'Isereal': 4}


In [7]:
def _populate_petroleum_products(raw_dict):
    product_name = set([(x['petroleum_product'],) for x in raw_dict])
    cursor.executemany("INSERT INTO petroleum_product(product_name) VALUES(?)",list(product_name))
    cursor.execute("SELECT * FROM petroleum_product")
    petroleum_products_with_ids = cursor.fetchall()
    #print(petroleum_products_with_ids)
    return {x[1]:x[0] for x in petroleum_products_with_ids}

product_name_with_id= _populate_petroleum_products(raw_dict)


In [8]:
print(product_name_with_id)

{'Light Diesel Oil': 1, 'Petrol': 2, 'Diesel': 3, 'Kerosene': 4, 'LPG in MT': 5, 'Aviation Turbine Fuel': 6, 'Furnace Oil': 7, 'Mineral Turpentine Oil': 8}


In [114]:
#print(raw_dict)

[{'year': '2014', 'petroleum_product': 'Petrol', 'sale': 283567, 'country': 'Saudi Arabia'}, {'year': '2014', 'petroleum_product': 'Diesel', 'sale': 901393, 'country': 'Saudi Arabia'}, {'year': '2014', 'petroleum_product': 'Kerosene', 'sale': 18628, 'country': 'Saudi Arabia'}, {'year': '2014', 'petroleum_product': 'Aviation Turbine Fuel', 'sale': 139404, 'country': 'Saudi Arabia'}, {'year': '2014', 'petroleum_product': 'Light Diesel Oil', 'sale': 0, 'country': 'Saudi Arabia'}, {'year': '2014', 'petroleum_product': 'Furnace Oil', 'sale': 0, 'country': 'Saudi Arabia'}, {'year': '2014', 'petroleum_product': 'LPG in MT', 'sale': 258299, 'country': 'Saudi Arabia'}, {'year': '2014', 'petroleum_product': 'Mineral Turpentine Oil', 'sale': 0, 'country': 'Saudi Arabia'}, {'year': '2014', 'petroleum_product': 'Petrol', 'sale': 183567, 'country': 'Isereal'}, {'year': '2014', 'petroleum_product': 'Diesel', 'sale': 201393, 'country': 'Isereal'}, {'year': '2014', 'petroleum_product': 'Kerosene', 'sal

In [45]:
# print([x['country'] for x in raw_dict])
# test = [(country_with_id[x['country']],) for x in raw_dict] #(country_with_id['Saudi Arabia'] = 3) 
# print(test)

In [9]:
normalized_sales_data = [
            (country_with_id[x['country']], product_name_with_id[x['petroleum_product']], x['year'], x['sale']) for x
            in raw_dict
                        ]
print(normalized_sales_data)

[(3, 2, '2014', 283567), (3, 3, '2014', 901393), (3, 4, '2014', 18628), (3, 6, '2014', 139404), (3, 1, '2014', 0), (3, 7, '2014', 0), (3, 5, '2014', 258299), (3, 8, '2014', 0), (4, 2, '2014', 183567), (4, 3, '2014', 201393), (4, 4, '2014', 38628), (4, 6, '2014', 0), (4, 1, '2014', 144677), (4, 7, '2014', 443673), (4, 5, '2014', 0), (4, 8, '2014', 0), (1, 2, '2014', 483567), (1, 3, '2014', 101393), (1, 4, '2014', 0), (1, 6, '2014', 12628), (1, 1, '2014', 144657), (1, 7, '2014', 0), (1, 5, '2014', 143273), (1, 8, '2014', 0), (2, 2, '2014', 0), (2, 3, '2014', 151393), (2, 4, '2014', 454393), (2, 6, '2014', 32628), (2, 1, '2014', 0), (2, 7, '2014', 0), (2, 5, '2014', 0), (2, 8, '2014', 401394), (3, 2, '2013', 123562), (3, 3, '2013', 701393), (3, 4, '2013', 28628), (3, 6, '2013', 239404), (3, 1, '2013', 1000), (3, 7, '2013', 0), (3, 5, '2013', 158299), (3, 8, '2013', 0), (4, 2, '2013', 0), (4, 3, '2013', 401493), (4, 4, '2013', 0), (4, 6, '2013', 28614), (4, 1, '2013', 184677), (4, 7, '2013

In [10]:
def _populate_sales_data(raw_dict):
    cursor.executemany("INSERT INTO petroleum_sales(country_id,product_id,year,sales) VALUEs(?,?,?,?)",raw_dict)

sales_data = _populate_sales_data(normalized_sales_data)


In [12]:
def _load_from_database():
    cudata annotation toolsrsor.execute("""
        select year, country.country, petroleum_product.product_name, sales 
        from petroleum_sales 
        join petroleum_product on petroleum_product.id = petroleum_sales.product_id
        join country on country.id = petroleum_sales.country_id
        """)
    return cursor.fetchall()

print(_load_from_database())

[(2014, 'Saudi Arabia', 'Petrol', 283567.0), (2014, 'Saudi Arabia', 'Diesel', 901393.0), (2014, 'Saudi Arabia', 'Kerosene', 18628.0), (2014, 'Saudi Arabia', 'Aviation Turbine Fuel', 139404.0), (2014, 'Saudi Arabia', 'Light Diesel Oil', 0.0), (2014, 'Saudi Arabia', 'Furnace Oil', 0.0), (2014, 'Saudi Arabia', 'LPG in MT', 258299.0), (2014, 'Saudi Arabia', 'Mineral Turpentine Oil', 0.0), (2014, 'Isereal', 'Petrol', 183567.0), (2014, 'Isereal', 'Diesel', 201393.0), (2014, 'Isereal', 'Kerosene', 38628.0), (2014, 'Isereal', 'Aviation Turbine Fuel', 0.0), (2014, 'Isereal', 'Light Diesel Oil', 144677.0), (2014, 'Isereal', 'Furnace Oil', 443673.0), (2014, 'Isereal', 'LPG in MT', 0.0), (2014, 'Isereal', 'Mineral Turpentine Oil', 0.0), (2014, 'USA', 'Petrol', 483567.0), (2014, 'USA', 'Diesel', 101393.0), (2014, 'USA', 'Kerosene', 0.0), (2014, 'USA', 'Aviation Turbine Fuel', 12628.0), (2014, 'USA', 'Light Diesel Oil', 144657.0), (2014, 'USA', 'Furnace Oil', 0.0), (2014, 'USA', 'LPG in MT', 143273.

In [15]:
def _task_iii():
    print("Total sales for each peroduct: \n")
    cursor.execute(
        """
        SELECT pp.product_name , sum(sales) total_sales 
        from petroleum_sales ps 
        join petroleum_product pp on pp.id  = ps.product_id 
        group by pp.product_name 
        order by sum(sales) desc
        """
    )
    result = cursor.fetchall()
    print("Country     Total sales")
    for record in result:
        print(record[0] + " " + str(record[1]) + '\n')
_task_iii()

Total sales for each peroduct: 

Country     Total sales
Kerosene 6056675.0

Light Diesel Oil 5835117.0

Aviation Turbine Fuel 5190359.0

Furnace Oil 5068525.0

Diesel 4944620.0

Petrol 3862204.0

LPG in MT 3388382.0

Mineral Turpentine Oil 2374917.0



In [122]:
def _task_iv():
    print("Top 3 countries that have the lowest sales: \ng")
    cursor.execute(
        """
        select country.country, sum(sales) total_sales
        from petroleum_sales ps 
        join country on country.id = ps.country_id 
        group by 1
        order by 2 ASC
        limit 3
        """
    )
    result = cursor.fetchall()
    print("Country     Total sales")
    for record in result:
        print(record[0] + " " + str(record[1]) + '\n')
        
_task_iv()

Top 3 countries that have the lowest sales: 
g
Country     Total sales
Isereal 7838552.0

USA 8673936.0

Russia 9461637.0



In [14]:
def _task_v():
    cursor.execute(
        """
        select year, petroleum_product.product_name, sales 
        from petroleum_sales 
        join petroleum_product on petroleum_product.id = petroleum_sales.product_id

        """
    )
    res = cursor.fetchall()
    print(res)

_task_v()

[(2014, 'Petrol', 283567.0), (2014, 'Diesel', 901393.0), (2014, 'Kerosene', 18628.0), (2014, 'Aviation Turbine Fuel', 139404.0), (2014, 'Light Diesel Oil', 0.0), (2014, 'Furnace Oil', 0.0), (2014, 'LPG in MT', 258299.0), (2014, 'Mineral Turpentine Oil', 0.0), (2014, 'Petrol', 183567.0), (2014, 'Diesel', 201393.0), (2014, 'Kerosene', 38628.0), (2014, 'Aviation Turbine Fuel', 0.0), (2014, 'Light Diesel Oil', 144677.0), (2014, 'Furnace Oil', 443673.0), (2014, 'LPG in MT', 0.0), (2014, 'Mineral Turpentine Oil', 0.0), (2014, 'Petrol', 483567.0), (2014, 'Diesel', 101393.0), (2014, 'Kerosene', 0.0), (2014, 'Aviation Turbine Fuel', 12628.0), (2014, 'Light Diesel Oil', 144657.0), (2014, 'Furnace Oil', 0.0), (2014, 'LPG in MT', 143273.0), (2014, 'Mineral Turpentine Oil', 0.0), (2014, 'Petrol', 0.0), (2014, 'Diesel', 151393.0), (2014, 'Kerosene', 454393.0), (2014, 'Aviation Turbine Fuel', 32628.0), (2014, 'Light Diesel Oil', 0.0), (2014, 'Furnace Oil', 0.0), (2014, 'LPG in MT', 0.0), (2014, 'Mine

In [19]:
cursor.execute(
        """
        select year, petroleum_product.product_name, sales 
        from petroleum_sales 
        join petroleum_product on petroleum_product.id = petroleum_sales.product_id

        """
    )
res = cursor.fetchall()

initial_year = sorted(set([x[0] for x in res]))
print(initial_year) 

input_dict = defaultdict(lambda:defaultdict(list))
output_dict = defaultdict(lambda:defaultdict(list))

for x in res:
    if x[2] != 0:
        input_dict[x[0]][x[1]].append(x[2])
        
    

print(input_dict[2007])

[2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014]
defaultdict(<class 'list'>, {'Petrol': [94732.0, 43122.0, 22232.0], 'Diesel': [27232.0, 12132.0, 13922.0, 9232.0], 'Kerosene': [987232.0, 92701.0, 9432.0], 'Aviation Turbine Fuel': [123356.0, 59232.0], 'Furnace Oil': [95732.0, 43187.0, 129232.0, 194732.0], 'LPG in MT': [84732.0, 527032.0, 29473.0], 'Light Diesel Oil': [29472.0, 294732.0, 294732.0], 'Mineral Turpentine Oil': [3612.0]})


In [23]:
l=[2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014]

def grouper(chunk_size, iterable):
    it = iter(iterable)
    while True:
        group = tuple(islice(it, None, chunk_size))
        if not group:
            break
        yield group
for i in grouper(4,l):
    print(i)

(2007, 2008, 2009, 2010)
(2011, 2012, 2013, 2014)


In [20]:
input_dict[2007]

defaultdict(list,
            {'Petrol': [94732.0, 43122.0, 22232.0],
             'Diesel': [27232.0, 12132.0, 13922.0, 9232.0],
             'Kerosene': [987232.0, 92701.0, 9432.0],
             'Aviation Turbine Fuel': [123356.0, 59232.0],
             'Furnace Oil': [95732.0, 43187.0, 129232.0, 194732.0],
             'LPG in MT': [84732.0, 527032.0, 29473.0],
             'Light Diesel Oil': [29472.0, 294732.0, 294732.0],
             'Mineral Turpentine Oil': [3612.0]})

In [21]:
for years in grouper(4,l):
    first_year = years[0]
    for single_year in years:
        for product,value in input_dict[single_year].items():
            if value != 0:
                output_dict[first_year][product].extend(value)
    print(output_dict[2007])
    formatted_string = """year       product            avg"""
    for year, product_dict in output_dict.items():
        formatted_string += "\n"
        for product, sales_list in product_dict.items():
            average_sales = sum(sales_list) / len(sales_list)
            formatted_string += str(year) + '-' + str(year + 4) + '  ' + product + '   ' + str(average_sales) + '\n'
print(formatted_string)



defaultdict(<class 'list'>, {'Petrol': [94732.0, 43122.0, 22232.0, 169474.0, 31145.0, 600189.0, 12375.0, 132424.0, 57424.0, 76123.0, 57844.0, 32642.0, 537826.0, 444123.0, 178242.0], 'Diesel': [27232.0, 12132.0, 13922.0, 9232.0, 103724.0, 33213.0, 213927.0, 243927.0, 223424.0, 61424.0, 51008.0, 33424.0, 23642.0, 33642.0], 'Kerosene': [987232.0, 92701.0, 9432.0, 298724.0, 51209.0, 119273.0, 332944.0, 128424.0, 32424.0, 32424.0, 823424.0, 128642.0, 32642.0, 23673.0, 823642.0], 'Aviation Turbine Fuel': [123356.0, 59232.0, 871234.0, 15922.0, 123673.0, 49087.0, 43724.0, 654242.0, 214249.0, 314246.0, 478242.0, 656422.0, 76523.0], 'Furnace Oil': [95732.0, 43187.0, 129232.0, 194732.0, 129574.0, 12355.0, 342927.0, 189474.0, 455524.0, 237812.0, 465424.0, 282424.0, 49876.0, 452642.0, 782642.0], 'LPG in MT': [84732.0, 527032.0, 29473.0, 19474.0, 139872.0, 125270.0, 17809.0, 20024.0, 84243.0, 237865.0, 98123.0, 823642.0, 82423.0, 12354.0, 246424.0], 'Light Diesel Oil': [29472.0, 294732.0, 294732.0, 

In [22]:
print(raw_dict)

[{'year': '2014', 'petroleum_product': 'Petrol', 'sale': 283567, 'country': 'Saudi Arabia'}, {'year': '2014', 'petroleum_product': 'Diesel', 'sale': 901393, 'country': 'Saudi Arabia'}, {'year': '2014', 'petroleum_product': 'Kerosene', 'sale': 18628, 'country': 'Saudi Arabia'}, {'year': '2014', 'petroleum_product': 'Aviation Turbine Fuel', 'sale': 139404, 'country': 'Saudi Arabia'}, {'year': '2014', 'petroleum_product': 'Light Diesel Oil', 'sale': 0, 'country': 'Saudi Arabia'}, {'year': '2014', 'petroleum_product': 'Furnace Oil', 'sale': 0, 'country': 'Saudi Arabia'}, {'year': '2014', 'petroleum_product': 'LPG in MT', 'sale': 258299, 'country': 'Saudi Arabia'}, {'year': '2014', 'petroleum_product': 'Mineral Turpentine Oil', 'sale': 0, 'country': 'Saudi Arabia'}, {'year': '2014', 'petroleum_product': 'Petrol', 'sale': 183567, 'country': 'Isereal'}, {'year': '2014', 'petroleum_product': 'Diesel', 'sale': 201393, 'country': 'Isereal'}, {'year': '2014', 'petroleum_product': 'Kerosene', 'sal